In [1]:
# Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
scaler_features = MinMaxScaler()
scaler_labels = MinMaxScaler()

# Deep Learning Pytorch Library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset, DataLoader
torch.cuda.is_available()
torch.cuda.empty_cache()
from numpy import savetxt
from numpy import loadtxt

In [2]:
Dimensions_load = loadtxt('ota1_dimensions.csv')
a  = []

In [3]:
print(Dimensions_load.shape)

(93, 28)


In [4]:
for i in range(93):
    # Load the iterative X filesS
    X_point_load = loadtxt(f'P{i}_X.csv')
    X_point_load = np.float16(X_point_load)

    # Choose the right dimension row and tile it to the X_point_load number of rows
    Pre_dim = Dimensions_load[i]
    Dim = np.tile(Pre_dim, (X_point_load.shape[0], 1))

    # Concatenate the two datasets along columns
    Concat = np.concatenate((X_point_load, Dim), axis = 1)

    #np.savetxt(f'X_and_Dims{i}.csv', Concat)

    if len(a) == 0:
        a = Concat
    else:
        a = np.concatenate((a,Concat), axis = 0)

In [5]:
b  = []
for i in range(93):
    Y_point_load = loadtxt(f'P{i}_Y.csv')
    Y_point_load = np.float16(Y_point_load)

    if len(b) == 0:
        b = Y_point_load
    else:
        b = np.concatenate((b,Y_point_load), axis = 0)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
a = np.float16(a)
b = np.float16(b)
a = pd.DataFrame(a)
a = a.dropna()
b = pd.DataFrame(b)
b = b.dropna()
a.to_numpy()
b.to_numpy()


a = scaler_features.fit_transform(a)
b = scaler_labels.fit_transform(b)


In [7]:
import joblib
joblib.dump(scaler_features, 'scaler_features.gz')
joblib.dump(scaler_labels, 'scaler_labels.gz')
#my_scaler = joblib.load('scaler.gz')

['scaler_labels.gz']

In [9]:
# Add delay line -------------------------
df_x_waves = pd.DataFrame(a[:,0:2])
df_x_dims = pd.DataFrame(a[:,2:30])

# One delay line with shift of 1
df_x_s1 = pd.DataFrame(a[:,0:2])
df_x_s1 = df_x_s1.shift(1)

# Second delay line with shift of 2
df_x_s2 = pd.DataFrame(a[:,0:2])
df_x_s2 = df_x_s2.shift(2)

# Y to dataframe
b = np.float16(b)
df_y = pd.DataFrame(b)

del a
del b

# Concatenate the columns and drop null values
all_df = pd.concat([df_x_waves, df_x_s1, df_x_s2, df_x_dims, df_y], axis = 1)

del df_x_waves, df_x_dims, df_x_s1, df_x_s2

all_df = all_df.sample(frac=0.7)
all_df = all_df.dropna()

# Convert to numpy
all = all_df.to_numpy()
X = all[:,0:34]
print(X.shape)
Y = all[:,34:all.shape[1]+1]
print(Y.shape)
# -----------------------------------------

(60296269, 34)
(60296269, 2)


In [10]:
np.savetxt('X_MLP_DelayLine2_all_dims.csv', X)
np.savetxt('Y_MLP_DelayLine2_all_dims.csv', Y)

In [ ]:
print(X.shape)
print(Y.shape)

(60295670, 34)
(60295670, 2)
